# PetitRADTRANS Emission Spectrum Retrieval Example
## Based on Molliere (2020)
https://arxiv.org/abs/2006.09394

Here we will outline the process of setting up a RetrievalDefinition object,
which is the class used to set up a petitRadTrans retrieval.
In general, you should create this object in the run_definition.py file,
which acts as the configuration file for the retrieval.
Once set up in the file, simply run <code>python retrieve.py</code>.
This program takes in the configuration, and uses pymultinest to find the best fit model, and produces standard retrieval outputs (best fit spectrum, corner plots, pressure-temperature profiles).

## Getting started
You should already have an installation of petitRadTrans on your machine, together with the opacity data files. Multinest and pymultinest are also required. 
Before you begin, ensure that you have ran <code>./make.sh</code>, which compiles the fortran code used in the retrieval for rebinning spectra.
For this example, several proprietary codes are used in the atmospheric model to solve disequilibrium chemistry and cloud condensation.
You can import other models from models.py, or write your own function, ensuring that it takes in the same set of inputs and outputs, which are documented in models.py.

In [1]:
# Let's start by importing everything we need.
import sys, os
# To not have numpy start parallelizing on its own
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np

from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc

# Import the class used to set up the retrieval.
from petitRADTRANS import RetrievalConfig
from petitRADTRANS import Retrieval

# Import Prior functions, if necessary.
from petitRADTRANS.retrieval.util import gaussian_prior

# Import atmospheric model function
# You could also write your own!
from petitRADTRANS.retrieval.models import emission_model_diseq

In [2]:
##########################
# Define the pRT run setup
##########################
# retrieve.py expects this object to be called RunDefinition.
RunDefinition = RetrievalConfig(retrieval_name = "retrieval_name", # Give a useful name for your retrieval outputs
                                run_mode = "retrieval",            # 'retrieval' to run, or 'evaluate' to make plots
                                AMR = True,                        # Adaptive mesh refinement, slower if True
                                plotting = False,                  # Only turn on to check your retrieval is working
                                scattering = True)                 # Add scattering for emission spectra clouds

## Data and Models
The data can be stored in either a text file (.txt, .dat) or in a .fits file.
If reading a text file, the first column must be the wavelength in micron, the second column the flux in W/m^2/micron for an emission spectrum, or in R_p^2/R_star^2 for a transmission spectrum. The final column must be the error on these data points.
If using a .fits file, it should contain a single HDU object named "SPECTRUM", with three fields: "WAVELENGTH", "FLUX" and "COVARIANCE".

The model_generating_function will take in all of the various retrieval parameters, and output the wavelength and spectrum as calculated by pRT. Several of these function are stored in models.py, but any function that takes the same arguments and returns the same outputs can be used. Later we will show an example of how to write such a function.

In [3]:
##################
# Read in Data
##################
RunDefinition.add_data('GRAVITY',                                        # Name of your dataset, typically the instrument name
                       'observations/HR8799e_Spectra.fits',              # File where the data is stored
                       model_generating_function = emission_model_diseq) # The atmospheric model used in this retrieval


## Parameters and Priors
Here we add all of the parameters used in the retrieval. 
Each parameter must be given a name, which matches the name used in the model function.
Parameters can be set to fixed or free. Fixed parameters must be given a value, while free parameters are given a function that transforms the unit hypercube generated by multinest into physical prior space. Various prior functions are stored in priors.py.

In [3]:
#################################################
# Add parameters, and priors for free parameters.
#################################################
RunDefinition.add_parameter(name = 'D_pl', free = False, value = 41.25*nc.pc)

# This run uses the model of Molliere (2020) for HR8799e
# The lambda function provide uniform priors.

# Log of the surface gravity in cgs units.
RunDefinition.add_parameter('log_g',True, 
                            transform_prior_cube_coordinate = \
                            lambda x : 2.+3.5*x)

# Planet radius in cm
RunDefinition.add_parameter('R_pl', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : ( 0.7+1.3*x)*nc.r_jup_mean)

# Interior temperature in Kelvin
RunDefinition.add_parameter('T_int', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : 300.+2000.*x,
                            value=0.0)

# Spline temperature structure parameters. T1 < T2 < T3
# As these priors depend on each other, they are implemented in the model function
RunDefinition.add_parameter('T3', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x,
                            value=0.0)
RunDefinition.add_parameter('T2', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x,
                            value=0.0)
RunDefinition.add_parameter('T1', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x)
# Optical depth model
# power law index in tau = delta * press_cgs**alpha
RunDefinition.add_parameter('alpha', True, \
                            transform_prior_cube_coordinate = \
                            lambda x :1.0+x)
# proportionality factor in tau = delta * press_cgs**alpha
RunDefinition.add_parameter('log_delta', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x) 
# Chemistry
# A 'free retrieval' would have each line species as a parameter
# Using a (dis)equilibrium model, we only supply bulk parameters.
# Carbon quench pressure
RunDefinition.add_parameter('log_pquench',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : -6.0+9.0*x)
# Metallicity
RunDefinition.add_parameter('Fe/H',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : -1.5+3.0*x)
# C/O ratio
RunDefinition.add_parameter('C/O',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 0.1+1.5*x)
# Clouds
# Based on an Ackermann-Marley (2001) cloud model
# Width of particle size distribution
RunDefinition.add_parameter('sigma_lnorm', True,
                            transform_prior_cube_coordinate = \
                            lambda x : 1.05 + 1.95*x) 
# Vertical mixing parameters
RunDefinition.add_parameter('log_kzz',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 5.0+8.*x) 
# Sedimentation parameter
RunDefinition.add_parameter('fsed',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 1.0 + 10.0*x)
# Data scaling
# If the luminosity of datasets does not agree, we can scale one or more of the datasets
# by a constant factor. 
# Scaling factor name must be the name of the Data class for
# the instrument, + "_scale_factor"                            
#RunDefinition.add_parameter('SPHERE_scale_factor',True,\
#                            transform_prior_cube_coordinate = \
#                            lambda x : 0.8 + 0.4 *x)

## Opacities
PetitRadTrans can be setup to use line by line opacities at R=10^6, or a lower resolution correlated-k mode (which we use here). The names of each species in these lists much match the names in the pRT opacity data files, derived from ExoMol line lists.

In addition to the line-by-line opacities, pRT also accounts for rayleigh scattering, CIA opaticies, and opacities for various cloud species.
If a cloud species is used, it will have a set of parameters added in order to retrieve the cloud abundance.
The parameters added depend on the model chosen. For this example, a parameter is added to scale the equilibrium cloud abundances. If the 'eq' parameter is turned off, parameters for the cloud abundance and the cloud base pressure are added.

In [4]:
#######################################################
# Define species to be included as absorbers
#######################################################
RunDefinition.set_rayleigh_species(['H2', 'He'])
RunDefinition.set_continuum_opacities(['H2-H2', 'H2-He'])
RunDefinition.set_line_species(['CH4', 'H2O', 'CO2', 'HCN', 'CO_all_iso', 'H2', 'H2S', 'NH3', 'PH3', 'Na', 'K'])
RunDefinition.add_cloud_species('Fe(c)_cd',eq = True,abund_lim = (-3.5,4.5))
RunDefinition.add_cloud_species('MgSiO3(c)_cd',eq = True,abund_lim = (-3.5,4.5))



## Running the retrieval
At this point, we are ready to run the retrieval! 
All we need to do is pass the RunDefinition we just created to the Retrieval class and call its run method.
There are a few additional parameters that can be adjusted, but the defaults should work well for almost all use cases.


In [ ]:
retrieval = Retrieval(rd,
                      output_dir = "",
                      sample_spec = True)


## Plotting
However, before running the retrieval, we should set up the plots.
Each parameter can be added to the corner plot, its label changed, and the values transformed to more nature units (eg, the planet radius in jupiter radii, rather than cm).
We can also set the bounds and scaling on the best fit spectrum plot and the limits for the P-T profile plot.
With this complete, our retrieval is ready to go.

In [6]:
##################################################################
# Define what to put into corner plot if run_mode == 'evaluate'
##################################################################
RunDefinition.parameters['R_pl'].plot_in_corner = True
RunDefinition.parameters['R_pl'].corner_label = r'$R_{\rm P}$ ($\rm R_{Jup}$)'
RunDefinition.parameters['R_pl'].corner_transform = lambda x : x/nc.r_jup_mean
RunDefinition.parameters['log_g'].plot_in_corner = True
RunDefinition.parameters['log_g'].corner_ranges = [2., 5.]
RunDefinition.parameters['log_g'].corner_label = "log g"
RunDefinition.parameters['fsed'].plot_in_corner = True
RunDefinition.parameters['log_kzz'].plot_in_corner = True
RunDefinition.parameters['log_kzz'].corner_label = "log Kzz"
RunDefinition.parameters['C/O'].plot_in_corner = True
RunDefinition.parameters['Fe/H'].plot_in_corner = True
RunDefinition.parameters['log_pquench'].plot_in_corner = True
RunDefinition.parameters['log_pquench'].corner_label = "log pquench"

"""for spec in RunDefinition.line_species:
    if 'all_iso' in spec:
        RunDefinition.parameters[spec].corner_label = 'CO'
    RunDefinition.parameters[spec].plot_in_corner = True
    RunDefinition.parameters[spec].corner_transform = lambda x : np.log10(x)
    RunDefinition.parameters[spec].corner_ranges = [-7,-1.0]
"""
for spec in RunDefinition.cloud_species:
    cname = spec.split('_')[0]
    RunDefinition.parameters['log_X_cb_'+cname].plot_in_corner = True
    RunDefinition.parameters['log_X_cb_'+cname].corner_label = cname

##################################################################
# Define axis properties of spectral plot if run_mode == 'evaluate'
##################################################################
RunDefinition.plot_kwargs["spec_xlabel"] = 'Wavelength (micron)'

RunDefinition.plot_kwargs["spec_ylabel"] = r'$(R_{\rm P}/R_*)^2$ (ppm)'
RunDefinition.plot_kwargs["y_axis_scaling"] = 1.0
RunDefinition.plot_kwargs["xscale"] = 'log'
RunDefinition.plot_kwargs["yscale"] = 'linear'
RunDefinition.plot_kwargs["resolution"] = 1500.

##################################################################
# Define from which observation object to take P-T
# in evaluation mode (if run_mode == 'evaluate'),
# add PT-envelope plotting options
##################################################################
RunDefinition.plot_kwargs["take_PTs_from"] = 'GRAVITY'
RunDefinition.plot_kwargs["temp_limits"] = [150, 3000]
RunDefinition.plot_kwargs["press_limits"] = [1e2, 1e-5]

Now that the plotting parameters are set up, we can start the retrieval using retrieval.run().
Once the retrieval is complete, we can use the plotAll() function to generate plots of the best fit spectrum, the pressure-temperature profile and the corner plots.

In [ ]:
retrieval.run()
retrieval.plotAll()

# Model Functions
In addition to setting up a retrieval with a standard mode, you may also want to write your own model to be used in the retrieval setup. This function will be passed as the <code>model_generating_function</code> argument when you <code>add_data</code> to the RetrievalConfig class. Here we will outline a simple, cloud-free transmission spectrum model using free retrieval chemistry and an isothermal P-T profile to demonstrate.

Four parameters are required. A pRT object which is created is from the Radtrans() class in pRT and a dictionary of parameters, which is created and stored by the RetrievalConfig class that we defined above. The remaining two parameters deal with plotting and the adaptive mesh refinement, which we won't use in this example.

In [2]:
# Lets start out by setting up a simple run definition
# We'll add the data AFTER we define the model function below
RunDefinitionSimple = RetrievalConfig(retrieval_name = "Free_Isothermal", 
                                      run_mode = "retrieval",            
                                      AMR = False,                        
                                      plotting = False,                  
                                      scattering = False) 
# Log of the surface gravity in cgs units.
RunDefinitionSimple.add_parameter('log_g',True, 
                            transform_prior_cube_coordinate = \
                            lambda x : 2.+3.5*x)

# Planet radius in cm
RunDefinitionSimple.add_parameter('R_pl', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : ( 0.7+1.3*x)*nc.r_jup_mean)

# Interior temperature in Kelvin
RunDefinitionSimple.add_parameter('Temperature', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : 300.+2000.*x,
                            value=0.0)
RunDefinitionSimple.set_rayleigh_species(['H2', 'He'])
RunDefinitionSimple.set_continuum_opacities(['H2-H2', 'H2-He'])

# Here we setup the line species for a free retrieval, setting the prior bounds with the abund_lim parameter
RunDefinitionSimple.set_line_species(['CH4', 'H2O', 'CO2','CO_all_iso'], free=True, abund_lim = (-6.0,6.0))


In [3]:
# Now we can define the atmospheric model we want to use
def retrieval_model_spec_iso(pRT_object, \
                             parameters, \
                             PT_plot_mode = False,
                             AMR = False):
    """
    retrieval_model_eq_transmission
    This model computes a transmission spectrum based on free retrieval chemistry
    and an isothermal temperature-pressure profile. 
    
    parameters
    -----------
    pRT_object : object
        An instance of the pRT class, with optical properties as defined in the RunDefinition.
    parameters : dict
        Dictionary of required parameters:
            Rstar : Radius of the host star [cm]
            log_g : Log of surface gravity
            R_pl : planet radius [cm]
            Temperature : Isothermal temperature [K]
            species : Abundances for each species used in the retrieval
            Pcloud : optional, cloud base pressure of a grey cloud deck.
    PT_plot_mode : bool
        Return only the pressure-temperature profile for plotting. Evaluate mode only.
    AMR : 
        Adaptive mesh refinement. Use the high resolution pressure grid around the cloud base.

    returns
    -------
    wlen_model : np.array
        Wavlength array of computed model, not binned to data [um]
    spectrum_model : np.array
        Computed transmission spectrum R_pl**2/Rstar**2
    """
    # Make the P-T profile
    pressures = pRT_object.press/1e6
    temperatures = parameters['Temperature'].value * np.ones_like(pressures)
       
    #or pp in parameters:
    #    print(parameters[pp].name,parameters[pp].value)

    # If in evaluation mode, and PTs are supposed to be plotted
    if PT_plot_mode:
        return pressures, temperatures
    
    # Make the abundance profiles
    abundances = {}
    msum = 0.0 # Check that the total massfraction of all species is <1
    for species in pRT_object.line_species:
        abundances[species] = parameters[species].value * np.ones_like(pressures)
        msum += parameters[species].value
    if msum > 1.0:
        return None, None
    abundances['H2'] = 0.766 * (1.0-msum) * np.ones_like(pressures)
    abundances['He'] = 0.234 * (1.0-msum) * np.ones_like(pressures)
    
    # Find the mean molecular weight in each layer
    MMW = calc_MMW(abundances)
    
    # Calculate the spectrum
    pRT_object.calc_transm(temperatures, \
                           abundances, \
                           10**parameters['log_g'].value, \
                           MMW, \
                           R_pl=parameters['R_pl'].value, \
                           P0_bar=100.)
                           # Keep P0_bar at 100. for now!
                           # Otherwise change maximum pressure
    wlen_model = nc.c/pRT_object.freq/1e-4
    spectrum_model = (pRT_object.transm_rad/parameters['Rstar'].value)**2.
    return wlen_model, spectrum_model

In [4]:
# Finally, we associate the model with the data, and we can run our retrieval!
RunDefinitionSimple.add_data('HST',                                         # Simulated HST data
                       'observations/spectrum_m0.5_co1.0nc_f0.1.txt',       # File where the data is stored
                       model_generating_function = retrieval_model_spec_iso) # The atmospheric model used in this retrieval


### Contact
If you need any additional help, don't hesitate to contact Evert Nasedkin (nasedkinevert@gmail.com).